In [60]:
%pip install pandas seaborn matplotlib 

You should consider upgrading via the '/Users/tenzin/Desktop/mycodes/nba-draft-ranker/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [61]:
from pathlib import Path

current_dir = Path.cwd()
df = pd.read_csv(current_dir.parent/ "college" / "college_stats_all_seasons.csv")
df.head()

,draft_year,player_slug,player_name,Season,Age,College,G,MP,Totals_FG,Totals_FGA,...,Totals_TOV,Totals_PF,Totals_PTS,Shooting_FG%,Shooting_3P%,Shooting_FT%,Per Game_MP,Per Game_PTS,Per Game_TRB,Per Game_AST
0,2000,martike01,Kenyon Martin,1996-97,19.0,CINCY,22,233.0,26,40,...,16.0,51.0,62,0.65,,0.313,10.6,2.8,3.4,0.5
1,2000,martike01,Kenyon Martin,1997-98,20.0,CINCY,30,858.0,124,198,...,52.0,109.0,298,0.626,0.0,0.476,28.6,9.9,8.9,1.4
2,2000,martike01,Kenyon Martin,1998-99,21.0,CINCY,33,900.0,142,248,...,67.0,109.0,334,0.573,0.0,0.562,27.3,10.1,6.9,1.5
3,2000,martike01,Kenyon Martin,1999-00,22.0,CINCY,31,909.0,221,389,...,56.0,71.0,585,0.568,0.286,0.684,29.3,18.9,9.7,1.4
4,2000,martike01,Kenyon Martin,Career,,,116,2900.0,513,875,...,191.0,340.0,1279,0.586,0.222,0.581,25.0,11.0,7.5,1.2


#### 1. Strip Column Names and Values

In [62]:
# Strip white spaces from column names
df.columns = df.columns.str.strip()

# Strip white spaces from column values
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

df.columns.tolist()


/var/folders/b9/7xp5dt9d5nd0hct357z94wgh0000gn/T/ipykernel_36347/3912004884.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


['draft_year',
 'player_slug',
 'player_name',
 'Season',
 'Age',
 'College',
 'G',
 'MP',
 'Totals_FG',
 'Totals_FGA',
 'Totals_3P',
 'Totals_3PA',
 'Totals_FT',
 'Totals_FTA',
 'Totals_ORB',
 'Totals_TRB',
 'Totals_AST',
 'Totals_STL',
 'Totals_BLK',
 'Totals_TOV',
 'Totals_PF',
 'Totals_PTS',
 'Shooting_FG%',
 'Shooting_3P%',
 'Shooting_FT%',
 'Per Game_MP',
 'Per Game_PTS',
 'Per Game_TRB',
 'Per Game_AST']

#### 2. Remove Unnecessary Columns

In [63]:
cols_to_drop = [
    "player_slug",
]

df = df.drop(columns=cols_to_drop)


#### 2. Remove NBA Career Rows


In [64]:
# Remove rows with feaeture "Season" = Career

df = df[df['Season'] != 'Career']
df.head()

,draft_year,player_name,Season,Age,College,G,MP,Totals_FG,Totals_FGA,Totals_3P,...,Totals_TOV,Totals_PF,Totals_PTS,Shooting_FG%,Shooting_3P%,Shooting_FT%,Per Game_MP,Per Game_PTS,Per Game_TRB,Per Game_AST
0,2000,Kenyon Martin,1996-97,19.0,CINCY,22,233.0,26,40,0,...,16.0,51.0,62,0.65,,0.313,10.6,2.8,3.4,0.5
1,2000,Kenyon Martin,1997-98,20.0,CINCY,30,858.0,124,198,0,...,52.0,109.0,298,0.626,0.0,0.476,28.6,9.9,8.9,1.4
2,2000,Kenyon Martin,1998-99,21.0,CINCY,33,900.0,142,248,0,...,67.0,109.0,334,0.573,0.0,0.562,27.3,10.1,6.9,1.5
3,2000,Kenyon Martin,1999-00,22.0,CINCY,31,909.0,221,389,2,...,56.0,71.0,585,0.568,0.286,0.684,29.3,18.9,9.7,1.4
5,2000,Stromile Swift,1998-99,19.0,LSU,16,319.0,45,110,1,...,46.0,43.0,121,0.409,0.125,0.6,19.9,7.6,4.3,0.3


#### 3. Retain only the last season of each player 

In [65]:
# Convert "Season" into a sortable numeric year
df["season_start"] = df["Season"].str[:4].astype(int)

# For each (player_name, draft_year, College), get the index of the latest season
idx_last = (
    df.groupby(["player_name", "draft_year", "College"])["season_start"]
      .idxmax()
)

# Extract only the last-season rows
last_season_df = df.loc[idx_last].reset_index(drop=True)

# Drop helper column
last_season_df = last_season_df.drop(columns=["season_start"])

# Drop column "Season"
last_season_df = last_season_df.drop(columns=["Season"])

In [66]:
## missing values report

def missing_report(dataframe):
    report = (
        dataframe.isna()
        .sum()
        .to_frame("missing_count")
    )
    report["total_rows"] = len(dataframe)
    report["missing_percent"] = (report["missing_count"] / report["total_rows"]) * 100
    report["non_missing_count"] = report["total_rows"] - report["missing_count"]

    return report[[
        "missing_count",
        "non_missing_count",
        "missing_percent"
    ]].sort_values("missing_percent", ascending=False)


missing_report(df)

,missing_count,non_missing_count,missing_percent
draft_year,0,2402,0.0
Totals_AST,0,2402,0.0
Per Game_AST,0,2402,0.0
Per Game_TRB,0,2402,0.0
Per Game_PTS,0,2402,0.0
Per Game_MP,0,2402,0.0
Shooting_FT%,0,2402,0.0
Shooting_3P%,0,2402,0.0
Shooting_FG%,0,2402,0.0
Totals_PTS,0,2402,0.0


_There is no missing values in the dataset, but we have scraped only if there is college stats Table from sports-reference.com_

In [71]:
last_season_df.sort_values(by=["draft_year", "Totals_PTS"], inplace=True)

last_season_df.to_csv(current_dir.parent/ "college"/'players_last_season.csv', index=False)

#### 4. Combine NBA Draft Combine and College Player Data

In [72]:
college_df = last_season_df.copy()
draft_df = pd.read_csv(current_dir.parent / "nba_combine.csv")

merged_df = pd.merge(
    draft_df,
    college_df,              
    left_on=["PLAYER_NAME", "SEASON"],
    right_on=["player_name", "draft_year"],
    how="left",                     # ⬅️ keep ALL rows from draft_df
    suffixes=("_DRAFT", "_COMBINE")
)

# Remove duplicates if any
merged_df = merged_df.drop_duplicates()

# Sort by season and draft order
merged_df = merged_df.sort_values(by=["SEASON", "OVERALL_PICK"])
merged_df.head()

,PERSON_ID,PLAYER_NAME,SEASON,ROUND_NUMBER,OVERALL_PICK,TEAM_NAME,ORGANIZATION,ORGANIZATION_TYPE,PLAYER_PROFILE_FLAG,POSITION,...,Totals_TOV,Totals_PF,Totals_PTS,Shooting_FG%,Shooting_3P%,Shooting_FT%,Per Game_MP,Per Game_PTS,Per Game_TRB,Per Game_AST
0,2030,Kenyon Martin,2000,1,1,Nets,Cincinnati,College/University,1,NaN,...,56.0,71.0,585.0,0.568,0.286,0.684,29.3,18.9,9.7,1.4
1,2031,Stromile Swift,2000,1,2,Grizzlies,Louisiana State,College/University,1,NaN,...,80.0,88.0,550.0,0.608,0.28,0.617,29.8,16.2,8.2,0.9
2,2032,Darius Miles,2000,1,3,Clippers,East St. Louis,High School,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2033,Marcus Fizer,2000,1,4,Bulls,Iowa State,College/University,1,NaN,...,77.0,103.0,844.0,0.582,0.357,0.732,33.6,22.8,7.7,1.1
4,2034,Mike Miller,2000,1,5,Magic,Florida,College/University,1,NaN,...,71.0,70.0,521.0,0.476,0.338,0.729,28.6,14.1,6.6,2.5


In [74]:
merged_df.to_csv(current_dir.parent/ "nba_draft_final.csv", index=False)